In [1]:
import dwavebinarycsp
import dimod
import neal
import numpy as np

In [2]:
def create_linear(vector):
    N = len(vector)
    linear_coeff = {}
    for k in range(N):
        key = 'x' + str(k+1)
        value = vector[k]
        linear_coeff[key] = value
    return linear_coeff

In [3]:
def create_quadratic(array):
    N, _ = np.shape(array)
    quadratic_coeff = {}
    for i in range(N):
        for j in range(i):
            key = ('x' + str(i+1) , 'x' + str(j+1))
            vaule = array[i][j]
            quadratic_coeff[key] = vaule
    return quadratic_coeff

In [9]:
v = np.loadtxt('value1000.txt')
# v = [1, 2, 3, 4]
N = len(v)
vector = np.random.randn(N)
for k in range(N):
    vector[k] = 4.0 * (v[k]**2 - v[k] * np.sum(v))
array = np.zeros((N, N))
for i in range(N):
    for j in range(N):
        array[i][j] = 8.0*v[i]*v[j]
bias = np.sum(v)**2

In [10]:
Linear = create_linear(vector)
Quad = create_quadratic(array)
#print(Linear)
#print(Quad)
print(bias)

116170825.2879295


In [11]:
import dimod
bqm = dimod.BinaryQuadraticModel(Linear, Quad, bias, 'BINARY')
len(bqm)

1000

run simulator

In [30]:
'''
run under local simulator with solver ExactSovler
'''
from dimod.reference.samplers import ExactSolver
sampler = ExactSolver()
response = sampler.sample(bqm)
print(response)

   x1 x2 x3 x4 energy num_oc.
4   0  1  1  0    0.0       1
14  1  0  0  1    0.0       1
5   1  1  1  0    4.0       1
6   1  0  1  0    4.0       1
12  0  1  0  1    4.0       1
15  0  0  0  1    4.0       1
2   1  1  0  0   16.0       1
7   0  0  1  0   16.0       1
8   0  0  1  1   16.0       1
13  1  1  0  1   16.0       1
3   0  1  0  0   36.0       1
9   1  0  1  1   36.0       1
1   1  0  0  0   64.0       1
11  0  1  1  1   64.0       1
0   0  0  0  0  100.0       1
10  1  1  1  1  100.0       1
['BINARY', 16 rows, 16 samples, 4 variables]


In [12]:
from dwave.cloud import Client
client = Client.from_config(
    token='')
client.get_solvers()

[StructuredSolver(id='DW_2000Q_6')]

In [31]:
'''
pure QA with limited scale
'''
from dwave.system import DWaveSampler, EmbeddingComposite
sampler = EmbeddingComposite(DWaveSampler())
sampleset = sampler.sample(bqm, num_reads=100)
print(sampleset)   # doctest: +SKIP

  x1 x2 x3 x4 energy num_oc. chain_.
0  1  1  0  1   16.0      50     0.5
2  0  0  1  1   16.0       1    0.25
3  1  1  0  1   16.0       1    0.25
1  1  0  1  1   36.0      48     0.5
['BINARY', 4 rows, 100 samples, 4 variables]


In [13]:
'''
https://docs.ocean.dwavesys.com/en/stable/docs_hybrid/intro/using.html
'''
from hybrid.reference.kerberos import KerberosSampler
solution = KerberosSampler().sample(bqm, max_iter=20, convergence=3)
solution.first.energy

0.0003195266181137413

In [53]:
s = solution.first.sample
p = np.zeros(N)
for k in range(N):
    key = 'x'+str(k+1)
    p[k] = 2*s[key] - 1
Sn = np.dot(p, v)**2
print(Sn)

0.0017286211270748785
